In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from dnn_classifier import DNNClassifer
from random import randint
from visualizer import *
from random import randint

2023-01-12 14:28:50.092880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-12 14:28:50.093078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-12 14:28:50.093092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
fm = FeatureManager()
fm.build_feature(data=data,lags=0)

In [5]:
take_profit_rate = 0.1
stop_loss_rate = 0.05
max_duration = 90


def prepare_trade_forward_data(data):
    
    data["take_profit"] = data["Close"]*(1+take_profit_rate)
    data["stop_loss"] = data["Close"]*(1-stop_loss_rate)
    data["cum_return_forward_0"] = data["returns"]
    data["decision_forward_0"] = 0
     
    for i in range(1,max_duration+1):
        
        data["High_forward_{}".format(i)] = data["High"].shift(-i)
        data["Low_forward_{}".format(i)] = data["Low"].shift(-i)
        data["cum_return_forward_{}".format(i)] = data["cum_return_forward_{}".format(i-1)]+data["returns"].shift(-i)

        str = "decision_forward_{}".format(i)
        data["random"] = [randint(0,100) for i in range(0,len(data))]
        
        #Temporarily set all open as closed from previous timeframe
        data[str] = 2

        # If position is still open, only stop-loss reached
        data.loc[
            (data["Low_forward_{}".format(i)]<data["stop_loss"])&
            (data["High_forward_{}".format(i)]<data["take_profit"])&
            (data["decision_forward_{}".format(i-1)]==0)
            ,str
            ] = -1

        # If position is still open, only take-profit reached
        data.loc[
            (data["Low_forward_{}".format(i)]>data["stop_loss"])&
            (data["High_forward_{}".format(i)]>data["take_profit"])&
            (data["decision_forward_{}".format(i-1)]==0)
            ,str
            ] = 1
        
        # If position is still open, both take-profit and stop-loss reached, odd say it was stoploss
        data.loc[
            (data["Low_forward_{}".format(i)]<data["stop_loss"])&
            (data["High_forward_{}".format(i)]>data["take_profit"])&
            (data["decision_forward_{}".format(i-1)]==0)&
            (data["random"]<50)
            ,str
            ] = -1
        
        # If position is still open, both take-profit and stop-loss reached, odd say it was take-profit
        data.loc[
            (data["Low_forward_{}".format(i)]<data["stop_loss"])&
            (data["High_forward_{}".format(i)]>data["take_profit"])&
            (data["decision_forward_{}".format(i-1)]==0)&
            (data["random"]>=50)
            ,str
            ] = 1
        
         # If position is still open, both take-profit and stop-loss not reached, leave it as open
        data.loc[
            (data["Low_forward_{}".format(i)]>data["stop_loss"])&
            (data["High_forward_{}".format(i)]<data["take_profit"])&
            (data["decision_forward_{}".format(i-1)]==0) 
            ,str
            ] = 0

        data.loc[(data["Low_forward_{}".format(i)]<data["stop_loss"])&(data["random"]>49),str] = -1
        data.loc[(data["High_forward_{}".format(i)]>data["take_profit"])&(data["random"]>49),str] = 1
    
prepare_trade_forward_data(fm.df)
fm.df.to_csv("../out/out.csv")

In [4]:
#results = pd.DataFrame(data = fm.df["decision_forward_3"].value_counts())
results = pd.DataFrame()
for i in range(1,max_duration+1):
    results["Forward {}".format(i)]= fm.df["decision_forward_{}".format(i)].value_counts()
results.to_csv("../out/results.csv")

In [ ]:
# Robot will based on current status, make a trade with a stoploss and profit. 
# We scan the future data and record profit/loss. It may look into smaller timeframe for better info. 
# The traded result, together with current status will be feeded to classifier model to learn
def action():
    ''' Return what to do in next interval 
        -1: Close as stoploss reached
         0: Keep
         1: Close as take profit reached
         2: Closed
    '''
    outcome = 0
    return outcome